In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import jaccard_score
import os
import time
from tqdm import tnrange, tqdm_notebook
# https://nbviewer.jupyter.org/github/kaushaltrivedi/bert-toxic-comments-multilabel/blob/master/toxic-bert-multilabel-classification.ipynb?source=post_page-------

In [2]:

WD = os.getcwd()
DATA_DIR = os.path.join(WD, 'data','mpst-movie-plot-synopses-with-tags','mpst_full_data.csv')

In [3]:
data = pd.read_csv(DATA_DIR)
data = data.drop(['synopsis_source'],axis=1)
data.shape

(14828, 5)

In [4]:
data.head()

,imdb_id,title,plot_synopsis,tags,split
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val


In [5]:
split = data['tags'].str.split(', ')
lens = split.str.len()


In [6]:
 np.concatenate(split)

array(['cult', 'horror', 'gothic', ..., 'anti war', 'murder',
       'christian film'], dtype='<U18')

In [7]:
temp_df = pd.DataFrame({'imdb_id': np.repeat(data['imdb_id'].values, lens), 
                        'category': np.concatenate(split),
                       'values': 1})

print(temp_df['category'].unique())
print(len(temp_df['category'].unique()))

temp_df = temp_df.pivot(index='imdb_id', columns='category', values='values').fillna(0).reset_index()



['cult' 'horror' 'gothic' 'murder' 'atmospheric' 'violence' 'romantic'
 'inspiring' 'stupid' 'feel-good' 'cruelty' 'dramatic' 'action' 'revenge'
 'sadist' 'queer' 'flashback' 'mystery' 'suspenseful' 'neo noir' 'prank'
 'psychedelic' 'tragedy' 'autobiographical' 'home movie'
 'good versus evil' 'depressing' 'realism' 'boring' 'haunting'
 'sentimental' 'paranormal' 'historical' 'storytelling' 'comedy' 'fantasy'
 'philosophical' 'adult comedy' 'cute' 'entertaining' 'bleak' 'humor'
 'plot twist' 'christian film' 'pornographic' 'insanity' 'brainwashing'
 'sci-fi' 'dark' 'claustrophobic' 'psychological' 'melodrama'
 'historical fiction' 'absurd' 'satire' 'alternate reality'
 'alternate history' 'comic' 'grindhouse film' 'thought-provoking'
 'clever' 'western' 'blaxploitation' 'whimsical' 'intrigue' 'allegory'
 'anti war' 'avant garde' 'suicidal' 'magical realism' 'non fiction']
71


In [8]:
data_separate = data.merge(temp_df, how='left', on='imdb_id')
data_separate.head()

,imdb_id,title,plot_synopsis,tags,split,absurd,action,adult comedy,allegory,alternate history,...,sentimental,storytelling,stupid,suicidal,suspenseful,thought-provoking,tragedy,violence,western,whimsical
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [95]:
# inbalance class ratio
1-(data_separate.iloc[:,5:].sum(axis=1)/71).mean()

0.9580105396338104

In [9]:
train_df = data_separate[data_separate['split'] == 'train']
val_df = data_separate[data_separate['split'] == 'val']
test_df = data_separate[data_separate['split'] == 'test']

train_df.shape, val_df.shape, test_df.shape

((9489, 76), (2373, 76), (2966, 76))

# BERT Modeling

In [97]:
import torch
from pytorch_transformers import *
from pytorch_transformers.modeling_bert import BertPreTrainedModel
from pytorch_transformers.optimization import AdamW

from torch.utils.data import Dataset, DataLoader
from torch.nn import BCEWithLogitsLoss
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [11]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
# bert_config = BertConfig.from_pretrained('bert-base-uncased')

In [23]:
class MPSTDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_seq_length, transform=None):
        self.dataframe = dataframe
        self.transform = transform
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.dataframe)
    
    def get_sample_features(self, sample):
        tokenized_sample = self.tokenizer.tokenize(sample)
        
        tokenized_sample = ["[CLS]"] + tokenized_sample[:self.max_seq_length-2] + ["[SEP]"]
    
        input_ids = self.tokenizer.convert_tokens_to_ids(tokenized_sample)
        segment_ids = [0] * len(input_ids)
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (self.max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding
        
        assert len(input_ids) == self.max_seq_length
        assert len(input_mask) == self.max_seq_length
        assert len(segment_ids) == self.max_seq_length
        
        return input_ids, input_mask, segment_ids


    def __getitem__(self, idx):
        sample = self.dataframe.iloc[idx]['plot_synopsis']
        label = self.dataframe.iloc[idx][5:]
        
        input_ids, input_mask, segment_ids = self.get_sample_features(sample)
        
        return torch.tensor(input_ids), torch.tensor(input_mask), torch.tensor(segment_ids), torch.tensor(label)
        

In [82]:
class BertForMultiLabelClassification(torch.nn.Module):
    def __init__(self, num_labels=71):
        super().__init__()
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = torch.nn.Dropout(0.1)
        self.layer = torch.nn.Linear(768,300)
        self.classifier = torch.nn.Linear(300, num_labels)
        self.batchnorm = torch.nn.BatchNorm1d(71)
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None,
                position_ids=None, head_mask=None):
        
        outputs = self.bert(input_ids, position_ids=position_ids, token_type_ids=token_type_ids,
                            attention_mask=attention_mask, head_mask=head_mask)

        # pooled output
        pooled_output = outputs[1]
        
        x = self.dropout(pooled_output)
        x = torch.nn.functional.relu(self.layer(x))
        logits = self.classifier(x)
        logits = self.batchnorm(logits)

        if labels is not None:
            loss_fct = BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1, self.num_labels))
            return loss, logits
        else:
            return logits
        
    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False
    
    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True

In [117]:
def train_model(dataloaders, model, optimizer, criterion, scheduler, num_epochs=2):
    since = time.time()
    step_sizes = {'train': len(dataloaders['train']), 
                     'valid': len(dataloaders['valid'])}
    
    weight = torch.tensor([0.1, 0.9])

    for epoch in tnrange(int(num_epochs), desc="Epoch"):
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0
            running_acc = 0
        
            for step, batch in enumerate(tqdm_notebook(dataloaders[phase], desc=phase)):
                batch = tuple(t.to(device) for t in batch)
                input_ids, input_mask, segment_ids, label_ids = batch
                
                logits = model(input_ids, segment_ids, input_mask)
                sigmoid = F.sigmoid(logits)

                loss = criterion(sigmoid, label_ids)
                
                label_weight = weight[label_ids.data.view(-1).long()].view_as(label_ids).to(device)
                weighted_loss = loss * label_weight
                weighted_loss_average = weighted_loss.mean()
                
                running_loss += weighted_loss_average.item()
                
                sigmoid_numpy = sigmoid.detach().cpu().numpy()
                labels_numpy = label_ids.detach().cpu().numpy()
    
                acc = jaccard_score(labels_numpy, sigmoid_numpy.round(), average='samples')
                running_acc += acc
        
                print(weighted_loss_average.item(), acc, sigmoid_numpy.round().sum(axis=1))
    
                if phase == 'train':
                    optimizer.zero_grad()
                    weighted_loss_average.backward()
                    optimizer.step()
#                     scheduler.step()
            
            if phase == 'train':
                train_loss = running_loss / step_sizes[phase]
                train_acc = running_acc / step_sizes[phase]
            else:
                valid_loss = running_loss / step_sizes[phase]
                valid_acc = running_acc / step_sizes[phase]
                
                print('Epoch [{}/{}] train loss: {:.4f} acc: {:.4f} ' 
              'valid loss: {:.4f} acc: {:.4f}'.format(
                epoch+1, num_epochs,
                train_loss, train_acc, 
                valid_loss, valid_acc))
            
    return model

In [118]:
train_ds = MPSTDataset(train_df, bert_tokenizer, 128)
train_dl = DataLoader(train_ds,batch_size=16, shuffle=True)

val_ds = MPSTDataset(val_df, bert_tokenizer, 256)
val_dl = DataLoader(val_ds,batch_size=16, shuffle=True)

dloaders = {'train':train_dl, 'valid':val_dl}

In [119]:
# EPOCHS = 10
# LEARNING_RATE = 3e-4
# ADAM_EPSILON = 1e-6
# WARMUP_STEPS = 0

# t_total= len(train_dl) * EPOCHS

# model = BertForMultiLabelClassification.from_pretrained("bert-base-uncased")
# model.to(device)

# criterion = BCEWithLogitsLoss()

# param_optimizer = list(model.named_parameters())
# no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
#     ]
# optimizer = AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE, eps=ADAM_EPSILON)
# scheduler = WarmupLinearSchedule(optimizer, warmup_steps=WARMUP_STEPS, t_total=t_total)

In [120]:
model = BertForMultiLabelClassification()
model.freeze_bert_encoder()

criterion = torch.nn.BCELoss(reduce=False)

optimizer = torch.optim.Adamax(model.parameters(), lr=0.001)


C:\Users\willi\AppData\Local\Continuum\miniconda3\envs\main\lib\site-packages\torch\nn\_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [121]:
start_time = time.time()
model = train_model(dloaders, model, optimizer,criterion, scheduler=None, num_epochs=10)
print('Training time: {:10f} minutes'.format((time.time()-start_time)/60))

C:\Users\willi\AppData\Local\Continuum\miniconda3\envs\main\lib\site-packages\torch\nn\functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.9000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
        ...,
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.9000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000]])
0.09277629107236862
0.019466108831568907 [37. 37. 36. 37. 32. 37. 34. 33. 37. 31. 39. 35. 40. 34. 32. 42.]
tensor([[0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.9000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
        ...,
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000]])
0.0955950915813446
0.03800519068393488 [32. 34. 41. 37. 37. 45. 34. 38. 28. 29. 34. 35. 32. 36. 3

KeyboardInterrupt: 

In [ ]:
# single_sample = train_ds[0]
# single_sample

In [34]:
model = BertForMultiLabelClassification()
model.freeze_bert_encoder()

criterion = BCEWithLogitsLoss()

optimizer = torch.optim.Adamax(model.parameters(), lr=0.001)


In [46]:
for i in range(100):
    single_sample = tuple(t.to(device) for t in single_sample)
    input_ids, input_mask, segment_ids, label_ids = single_sample
    
    logits = model(input_ids.unsqueeze(0), segment_ids.unsqueeze(0), input_mask.unsqueeze(0))
    
    loss = criterion(logits.view(-1, 71), label_ids.view(-1,71))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(loss)

tensor(0.0382, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0359, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0321, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0316, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0307, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0257, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0274, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0278, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0286, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0272, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0264, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0244, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0252, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0268, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0251, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor(0.0240, grad_fn=<BinaryCrossEntropyWithLogitsBac